In [1]:
import tensorflow as tf

from tensorflow.python.framework import ops

ops.reset_default_graph()

E:\Anaconda3_5_0_0\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
feature_clum = []
for i in range(28*28):
    feature_clum.append(tf.feature_column.numeric_column(key=( "pixel%d" % i )))

In [3]:
runConfig = tf.estimator.RunConfig(model_dir="./model")

digit_class = tf.estimator.DNNClassifier(feature_columns=feature_clum,
                                         hidden_units=[10,10],n_classes = 10,config=runConfig)

INFO:tensorflow:Using config: {'_model_dir': './model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002ECDCE6D240>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [4]:
DATA_TYPE = [[0]] * (28*28 +1)
CSV_COLUMN_NAMES = ["pixel%d" % i for i in range(28 * 28)]
CSV_COLUMN_NAMES.insert(0,"label")

In [5]:
def _parse_train_line(line):
    field = tf.decode_csv(line,record_defaults=DATA_TYPE)
    
    features = dict(zip(CSV_COLUMN_NAMES,field))
    
    label = features.pop("label")
    
    return features,label

In [6]:
def csv_load(csv_path,skip,batch_size,parse_fn):
    dataset_digit = tf.data.TextLineDataset(csv_path).skip(skip)
    tf.logging.info("run csv load")
    dataset_digit = dataset_digit.map(parse_fn)
    dataset_digit = dataset_digit.shuffle(1000).batch(batch_size)
    return dataset_digit.make_one_shot_iterator().get_next()
    

In [7]:
def input_fn_train(batch_size):
    return csv_load("../digit-recognizer/train.csv",10001,batch_size,_parse_train_line)

In [8]:
digit_class.train(input_fn=lambda:input_fn_train(100),steps=10000)

INFO:tensorflow:run csv load
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./model\model.ckpt.
INFO:tensorflow:loss = 7870.9526, step = 0
INFO:tensorflow:global_step/sec: 6.19121
INFO:tensorflow:loss = 227.85715, step = 100 (16.153 sec)
INFO:tensorflow:global_step/sec: 7.63954
INFO:tensorflow:loss = 232.88184, step = 200 (13.090 sec)
INFO:tensorflow:global_step/sec: 7.57054
INFO:tensorflow:loss = 223.49979, step = 300 (13.209 sec)
INFO:tensorflow:Saving checkpoints for 320 into ./model\model.ckpt.
INFO:tensorflow:Loss for final step: 229.04874.


In [9]:
def input_fn_eval(batch_size):
    test_digit_dataset = tf.data.TextLineDataset("../digit-recognizer/train.csv").skip(1).take(10000).map(_parse_train_line).shuffle(1000).batch(batch_size)
    return test_digit_dataset.make_one_shot_iterator().get_next()

eval_result = digit_class.evaluate(input_fn=lambda:input_fn_eval(100))
print(eval_result)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-07-02:30:59
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model\model.ckpt-320
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-07-02:31:15
INFO:tensorflow:Saving dict for global step 320: accuracy = 0.1509, average_loss = 2.2365792, global_step = 320, loss = 223.65793
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 320: ./model\model.ckpt-320
{'accuracy': 0.1509, 'average_loss': 2.2365792, 'loss': 223.65793, 'global_step': 320}
